In [3]:
# import libraries

import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

# create a list of feature names

FEATURES = []
for i in range(1, 769):
  FEATURES.append('feature_' + str(i))
# create a list of label names

LABELS = ['label_1', 'label_2', 'label_3', 'label_4']

# read the data

df_train = pd.read_csv('../Dataset/layer_10_train.csv')
df_valid = pd.read_csv('../Dataset/layer_10_valid.csv')
df_test = pd.read_csv('../Dataset/layer_10_test.csv')

# store the data in a dictionary for each label

data_dict = dict()

for label in LABELS:
  data_dict[label] = dict()

  data_dict[label]['x_train'] = df_train[df_train[label].notna()][FEATURES].values
  data_dict[label]['y_train'] = df_train[df_train[label].notna()][label].values
  data_dict[label]['x_valid'] = df_valid[df_valid[label].notna()][FEATURES].values
  data_dict[label]['y_valid'] = df_valid[df_valid[label].notna()][label].values
  data_dict[label]['x_test'] = df_test[FEATURES].values


# create models for each label

model_dict = dict()

model_dict[LABELS[0]] = LogisticRegression(C=1)
model_dict[LABELS[1]] = KNeighborsClassifier(n_neighbors = 1, p = 2)
model_dict[LABELS[2]] = SVC(C = 100, kernel = 'rbf')
model_dict[LABELS[3]] = SVC(C=100, kernel='rbf')

df_test_pred = pd.DataFrame()

for label in LABELS:

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(data_dict[label]['x_train'])
  X_val_scaled = scaler.transform(data_dict[label]['x_valid'])
  X_test_scaled = scaler.transform(data_dict[label]['x_test'])

  # pca = PCA(n_components=0.98, svd_solver='full')
  # X_train_pca = pca.fit_transform(X_train_scaled)
  # X_val_pca = pca.transform(X_val_scaled)
  # X_test_pca = pca.transform(X_test_scaled)

  model_dict[label].fit(X_train_scaled, data_dict[label]['y_train'])
  data_dict[label]['y_pred'] = model_dict[label].predict(X_val_scaled)
  print('accuracy_score for {}: '.format(label), accuracy_score(data_dict[label]['y_valid'], data_dict[label]['y_pred']))

  data_dict[label]['y_pred_test'] = model_dict[label].predict(X_test_scaled)
  df_test_pred[label] = data_dict[label]['y_pred_test']

  data_dict[label]['y_pred_train'] = model_dict[label].predict(X_train_scaled)

  pd.DataFrame(data_dict[label]['y_pred_train']).to_csv('layer_10_train_pred_{}.csv'.format(label), index=False)


/home/akila-i/.pyenv/versions/3.9.9/envs/ml-py3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


accuracy_score for label_1:  0.964
accuracy_score for label_2:  0.9239130434782609
accuracy_score for label_3:  1.0
accuracy_score for label_4:  0.9813333333333333


In [4]:
print(len(data_dict['label_1']['y_pred_train']))
print(len(data_dict['label_2']['y_pred_train']))
print(len(data_dict['label_3']['y_pred_train']))
print(len(data_dict['label_4']['y_pred_train']))


28520
28040
28520
28520
